<a href="https://colab.research.google.com/github/GiorgioMB/UniversityProjects/blob/BERT-Project/Financial_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The scope of this model is to analyse the sentiment of financial headlines, tweets, or single phrases

To make the following model work, please download the pretrained model.zip here: https://drive.google.com/uc?export=download&id=1xXjXfsOpMYkDVb4_XccRMgxu5eqSvNkh

For unknown reasons, the following doesn't work on Safari

In [ ]:
!pip install transformers
!pip install torch torchvision torchaudio
!pip install requests
!pip install selenium
!pip install --upgrade google-colab
!pip install numpy

When prompted, please locate and select the downloaded zip file

In [ ]:
from google.colab import files
import zipfile
import io


uploaded = files.upload()


uploaded_file_name = list(uploaded.keys())[0]
extraction_path = 'extracted_model/'
with zipfile.ZipFile(uploaded_file_name, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print("File uploaded and extracted successfully.")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
model_folder_path = 'extracted_model/models'

tokenizer = AutoTokenizer.from_pretrained(model_folder_path)
model = AutoModelForSequenceClassification.from_pretrained(model_folder_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

sentiment_labels = ['Negative', 'Positive', 'Neutral']


# Relevant section

In [ ]:
while True:
    user_input = input("Enter text for sentiment analysis (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    tokenized_input = tokenizer(user_input, padding=True, truncation=True, return_tensors='pt')
    input_ids = tokenized_input['input_ids'].to(device)
    attention_mask = tokenized_input['attention_mask'].to(device)

    with torch.no_grad():
        model.eval()
        outputs = model(input_ids, attention_mask=attention_mask)
        predicted_probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_label = predicted_probabilities.argmax(dim=-1).item()
        confidence_score = predicted_probabilities[0][predicted_label].item()

        predicted_sentiment = sentiment_labels[predicted_label]
        print(f"Predicted Sentiment: {predicted_sentiment}")
        print(f"Confidence Score: {confidence_score:.4f}")
